<a href="https://colab.research.google.com/github/FudgeSato/Quant-Backtesting-Monte-Carlo/blob/main/Csv_Upload_General_Portfolio_Backtester_and_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
tickers = ["NVDA", "TSM", "BRK-B", "BX", "KKR", "BAM", "APO", "LLY", "BAESY",
           "TSLA", "NFLX", "EQIX", "DLR", "WMT", "COST", "KO",
           "VST", "LVMUY", "LMT", "ULTA", "GOLD"]

In [61]:
# @title
import yfinance as yf
import pandas as pd

def get_fundamental_data(tickers):
    data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Extract fundamental data
        market_cap = info.get("marketCap", None)
        revenue = info.get("totalRevenue", None)
        pe_ratio = info.get("trailingPE", None)
        roe = info.get("returnOnEquity", None)
        debt_equity = info.get("debtToEquity", None)
        dividend_yield = info.get("dividendYield", None)
        beta = info.get("beta", None)

        data.append([ticker, market_cap, revenue, pe_ratio, roe, debt_equity, dividend_yield, beta])

    # Convert to DataFrame
    columns = ["Ticker", "Market Cap", "Revenue", "P/E Ratio", "ROE", "Debt/Equity", "Dividend Yield", "Beta"]
    df = pd.DataFrame(data, columns=columns)
    return df

# Example tickers from your portfolio

df = get_fundamental_data(tickers)
print(df)  # Display results

df.to_csv("fundamental_data.csv", index=False)  # Save data to CSV


   Ticker     Market Cap       Revenue   P/E Ratio      ROE  Debt/Equity  \
0    NVDA  2816491913216  1.304970e+11   39.261906  1.19177       12.946   
1     TSM   897947402240  2.894308e+12   25.200874  0.30036       24.288   
2   BRK-B  1128031191040  3.714330e+11   12.672966  0.14653       20.066   
3      BX   178155896832  1.266259e+10   40.272728  0.28947       68.543   
4     KKR   105219817472  2.642180e+10   34.762196  0.08152       82.822   
5     BAM    79465504768           NaN   37.875000  0.20175        6.986   
6     APO    78406770688  2.588800e+10   18.750341  0.22670      103.066   
7     LLY   738753773568  4.504270e+10   70.420370  0.84264      244.616   
8   BAESY    65744543744  2.631200e+10   26.348349  0.18142       87.594   
9    TSLA   724714127360  9.769000e+10  109.907320  0.10420       18.489   
10   NFLX   397805453312  3.900097e+10   46.944977  0.38435       72.726   
11   EQIX    81881522176  8.737000e+09   98.971760  0.06246      139.913   
12    DLR   

In [62]:
# @title
import pandas as pd

def calculate_weights(df):
    # Ensure no division errors or NaN values
    df = df.dropna(subset=["ROE", "Market Cap", "Debt/Equity"]).copy()  # Create an explicit copy

    # Convert percentages to decimals using .loc
    df.loc[:, "ROE"] = df["ROE"].astype(float)
    df.loc[:, "Market Cap"] = df["Market Cap"].astype(float)
    df.loc[:, "Debt/Equity"] = df["Debt/Equity"].astype(float)

    # Growth & Profitability Weighting (Strategy 1)
    df.loc[:, "Growth_Weight"] = df["ROE"] * df["Market Cap"]
    df.loc[:, "Growth_Weight"] /= df["Growth_Weight"].sum()

    # Quality & Low Leverage Weighting (Strategy 2)
    df.loc[:, "Quality_Weight"] = 1 / (1 + df["Debt/Equity"])
    df.loc[:, "Quality_Weight"] /= df["Quality_Weight"].sum()

    # Final Weight: Blend of both strategies
    df.loc[:, "Final_Weight"] = (df["Growth_Weight"] + df["Quality_Weight"]) / 2
    df.loc[:, "Final_Weight"] /= df["Final_Weight"].sum()

    return df[["Ticker", "Final_Weight"]]

# Load your fundamental data
file_path = "fundamental_data.csv"  # Make sure this file is in the same directory
df = pd.read_csv(file_path)

# Calculate the smart beta weights
weighted_df = calculate_weights(df)

# Save the new portfolio weights to CSV without sorting
weighted_df.to_csv("smart_beta_portfolio.csv", index=False)

# Load the portfolio data
portfolio_df = pd.read_csv("smart_beta_portfolio.csv")

# Sort by weight in descending order
sorted_portfolio = portfolio_df.sort_values(by=["Final_Weight"], ascending=False)

# Display the sorted portfolio
print(sorted_portfolio)

# Save the sorted portfolio to a new CSV file (optional)
sorted_portfolio.to_csv("sorted_portfolio.csv", index=False)

   Ticker  Final_Weight
0    NVDA      0.377642
5     BAM      0.112026
7     LLY      0.061892
1     TSM      0.060162
2   BRK-B      0.057379
20   GOLD      0.052818
9    TSLA      0.052362
14   COST      0.039491
13    WMT      0.027428
10   NFLX      0.026291
17  LVMUY      0.020696
3      BX      0.017522
15     KO      0.016020
19   ULTA      0.012027
4     KKR      0.011333
12    DLR      0.011098
8   BAESY      0.011080
18    LMT      0.010986
6     APO      0.010146
11   EQIX      0.006743
16    VST      0.004859


In [63]:
# @title
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import csv

# Replace user input for tickers and weights with CSV reading
tickers = []
weights = []

with open('/content/sorted_portfolio.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row if it exists
    for row in reader:
        tickers.append(row[0])  # Assuming ticker is in the first column
        weights.append(float(row[1]))  # Assuming weight is in the second column and needs to be converted to float

# User selects benchmarks
print("Enter benchmark tickers (comma-separated, e.g., SPY,QQQ,BND,AOR,VTI,VT) or press Enter for none:")
benchmark_input = input().strip()
benchmarks = [ticker.strip() for ticker in benchmark_input.split(",")] if benchmark_input else []

all_tickers = tickers + benchmarks

# Adjust the end date to a date in the past
data = yf.download(all_tickers, start="2021-01-01", end="2025-03-11")["Close"]
data = data.ffill().bfill()  # Handle missing values

# Check if 'Ticker' column exists and use it if necessary
if 'Ticker' in data.columns:
    data = data.set_index('Ticker').T  # Transpose to have tickers as columns

# Step 2: Compute Returns and Portfolio Performance
returns = data.pct_change().dropna()
portfolio_returns = returns[tickers] @ weights
cumulative_returns = (1 + portfolio_returns).cumprod() * 10000  # Starting at $100

# Compute benchmark cumulative returns
if benchmarks:
    benchmark_cumulative_returns = (1 + returns[benchmarks]).cumprod() * 10000

# Step 4: Risk/Reward Analysis
annualized_return = (portfolio_returns.mean() * 252)
volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratio = (annualized_return - 0.02) / volatility  # 2% risk-free rate

# Step 5: Interactive Visualization
fund_name = input("Enter the name of your portfolio: ")
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name=f'{fund_name}', line=dict(width=3)))  # Line width changed here
if benchmarks:
    for benchmark in benchmarks:
        fig.add_trace(go.Scatter(x=benchmark_cumulative_returns.index, y=benchmark_cumulative_returns[benchmark], mode='lines', name=benchmark))
# Add annotations for key metrics
fig.add_annotation(
    x=0.05,  # Adjust x position as needed
    y=0.95,  # Adjust y position as needed
    xref="paper",  # Relative to the plot area
    yref="paper",  # Relative to the plot area
    text=f"Annualized Return: {annualized_return:.2%}<br>Volatility: {volatility:.2%}<br>Sharpe Ratio: {sharpe_ratio:.2f}",
    showarrow=False,  # No arrow
    font=dict(size=12, color="white"),  # Adjust font as needed
    bgcolor="rgba(0, 0, 0, 0.5)",  # Semi-transparent background
    bordercolor="white",  # Border color
    borderwidth=1  # Border width
)

fig.update_layout(
    title={
        'text': f"{fund_name} Portfolio Performance vs Benchmarks",
        'subtitle': {'text': f"Your portfolio's performance over time ({tickers})"}
    },
    xaxis_title="Date", yaxis_title="Value ($)", template="plotly_dark"
)
fig.show()


# Print key metrics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Volatility: {volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Enter benchmark tickers (comma-separated, e.g., SPY,QQQ,BND,AOR,VTI,VT) or press Enter for none:
SPY


[*********************100%***********************]  22 of 22 completed


Enter the name of your portfolio: Titan Ashura


Annualized Return: 36.64%
Volatility: 28.08%
Sharpe Ratio: 1.23


In [64]:
# @title
# prompt: calculate and plot the 50 and 200 day SMA of the portfolio, scale the SMA lines to the NAV of the portfolio, and only plot the scaled lines against the NAV

# Calculate 50-day and 200-day SMAs
cumulative_returns.index = pd.to_datetime(cumulative_returns.index)
sma50 = cumulative_returns.rolling(window=50).mean()
sma200 = cumulative_returns.rolling(window=200).mean()

# Scale SMA lines to NAV
sma50_scaled = sma50 / cumulative_returns.iloc[-1] * cumulative_returns.iloc[-1]
sma200_scaled = sma200 / cumulative_returns.iloc[-1] * cumulative_returns.iloc[-1]


# Plotting
fig = go.Figure()

# Add NAV
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name=f'{fund_name}', line=dict(width=3)))

# Add scaled SMAs
fig.add_trace(go.Scatter(x=sma50_scaled.index, y=sma50_scaled, mode='lines', name='50-Day SMA (Scaled)', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=sma200_scaled.index, y=sma200_scaled, mode='lines', name='200-Day SMA (Scaled)', line=dict(color='purple')))


# Update layout (similar to your existing layout)
fig.update_layout(
    title={
        'text': f"{fund_name} Portfolio Performance with SMAs",
        'subtitle': {'text': f"Your portfolio's performance over time ({tickers})"}
    },
    xaxis_title="Date", yaxis_title="Value ($)", template="plotly_dark"
)
fig.show()


In [65]:
# @title
import plotly.graph_objects as go
# Step 3: Monte Carlo Simulation with TensorFlow for GPU Acceleration
num_simulations = 10000  # Scalable to 1 million
num_days = len(portfolio_returns)
batch_size = 10000  # Run simulations in batches to avoid VRAM overload

# Limit VRAM usage to 10GB
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    except RuntimeError as e:
        print(e)

# Convert data to TensorFlow tensors
portfolio_returns_tf = tf.convert_to_tensor(portfolio_returns.values, dtype=tf.float32)

def monte_carlo_simulation():
    random_returns = tf.random.shuffle(portfolio_returns_tf, seed=None)
    return tf.math.cumprod(1 + random_returns, axis=0) * 10000

# Run Monte Carlo in batches if VRAM is exhausted
simulated_nav = []
for _ in range(num_simulations // batch_size):
    batch_result = tf.map_fn(lambda _: monte_carlo_simulation(), tf.range(batch_size), dtype=tf.float32)
    simulated_nav.append(batch_result.numpy())

simulated_nav = np.concatenate(simulated_nav, axis=1)

# Step 6: 1-Year Future Monte Carlo Simulation
future_days = 252

# Generate future random returns based on historical distribution
future_simulations = np.random.choice(portfolio_returns, size=(future_days, num_simulations))
future_cumulative_returns = np.cumprod(1 + future_simulations, axis=0) * 10000

# Calculate max drawdown
def max_drawdown(returns):
    cumulative = np.cumprod(1 + returns, axis=0)
    peak = np.maximum.accumulate(cumulative, axis=0)
    drawdown = (cumulative - peak) / peak
    return drawdown.min(axis=0)

future_max_drawdowns = max_drawdown(future_simulations)

# Calculate percentiles of returns
percentiles = np.percentile(future_cumulative_returns[-1, :], [25, 50, 75, 90])
mean_return = np.mean(future_cumulative_returns[-1, :])

# Interactive plot using Plotly
fig = go.Figure()
for i in range(100):  # Plot only 100 paths for visibility
    fig.add_trace(go.Scatter(x=list(range(future_days)), y=future_cumulative_returns[:, i],
                             mode='lines', line=dict(width=1, color='red'), opacity=0.1, showlegend=False))

fig.update_layout(
    title=f"{fund_name} Portfolio: 1-Year Future Monte Carlo Simulation",
    xaxis_title="Days into the Future",
    yaxis_title="Future NAV",
    template="plotly_dark"
)
fig.show()

# Print statistics
print(f"Max Drawdown: {np.min(future_max_drawdowns):.2%}")
print(f"25th Percentile Return: {percentiles[0]:.2f}")
print(f"50th Percentile (Median) Return: {percentiles[1]:.2f}")
print(f"Mean Return: {mean_return:.2f}")
print(f"75th Percentile Return: {percentiles[2]:.2f}")
print(f"90th Percentile Return: {percentiles[3]:.2f}")


Max Drawdown: -57.72%
25th Percentile Return: 11467.80
50th Percentile (Median) Return: 13915.19
Mean Return: 14449.41
75th Percentile Return: 16810.71
90th Percentile Return: 19982.46


In [66]:
# @title
portfolio = pd.DataFrame({'Ticker': tickers, 'Weight': weights})
portfolio['Allocation'] = portfolio['Weight'] * 10000
print(portfolio[['Ticker', 'Allocation']])  # Display tickers and allocations

   Ticker   Allocation
0    NVDA  3776.423568
1     BAM  1120.256923
2     LLY   618.920637
3     TSM   601.623189
4   BRK-B   573.788267
5    GOLD   528.182697
6    TSLA   523.616028
7    COST   394.906832
8     WMT   274.281722
9    NFLX   262.909941
10  LVMUY   206.963193
11     BX   175.217879
12     KO   160.198975
13   ULTA   120.265773
14    KKR   113.333110
15    DLR   110.975480
16  BAESY   110.798450
17    LMT   109.862216
18    APO   101.462452
19   EQIX    67.427348
20    VST    48.585319
